# Simulation setup

In [ ]:
import numpy as np
from brian2 import *

# overall model parameters

N = 400
Ni = 100
n_cl = 5
p_cl = np.ones(n_cl)/n_cl
Ns = np.round(N/n_cl) * np.ones(n_cl, dtype=int)  #np.random.multinomial(N, p_cl)
Cs = np.hstack([i * np.ones(Ns[i],dtype=np.int32) for i in range(n_cl)])

scale = N/4000.
duration = 2*second

mod = 0.83
mod2 = 0.96
Vt = 1. # firing threshold
Vr = 0. # reset potential

model_eqs = '''dv/dt= 1.0/tau * (myu - v) + I_syn : 1
               myu : 1
               I_syn =  - I_syn_i + I_syn_e : Hz
               I_syn_e = x_e/(tau2_e-tau1_e) : Hz
               dx_e/dt = -(normalization_e*y_e+x_e)*invtau1_e : 1
               dy_e/dt = -y_e*invtau2_e : 1
               I_syn_i = x_i/(tau2_i-tau1_i) : Hz
               dx_i/dt = -(normalization_i*y_i+x_i)*invtau1_i : 1
               dy_i/dt = -y_i*invtau2_i : 1
            '''


# excitation-specific parameters

myueMax = 1.2 *mod2
myueMin = 1.1 *mod2

taue = 15. * ms * mod
tau1 = 1.*ms *mod
tau2e = 3. * ms * mod

refractory = 5 * ms

type_type_conn = 0.2 + 0*np.sqrt(0.01) * np.random.normal( size=(n_cl, n_cl) )
z = np.zeros(type_type_conn.shape)
type_type_conn = np.where(type_type_conn >= z, type_type_conn, z)

type_type_strn = 0.024 + 0.0001 * np.random.normal( size=(n_cl, n_cl) )
z = np.zeros(type_type_strn.shape)
type_type_strn = np.where(type_type_strn >= z, type_type_strn, z) / np.sqrt(scale)

type_tau = 15. * mod * np.ones(n_cl) * ms #np.random.uniform(low=10, high=20, size=(n_cl,)) * msecond
type_tau1 = 1. * mod * np.ones(n_cl) * ms #np.random.uniform(low=10, high=20, size=(n_cl,)) * msecond
type_tau2 = 3. * mod * np.ones(n_cl) * ms #np.random.uniform(low=10, high=20, size=(n_cl,)) * msecond

type_Vt = Vt * np.ones(n_cl) #np.random.uniform(low=-70, high=-55, size=(n_cl,)) * mvolt
type_Vr = Vr * np.ones(n_cl)  #typeVt - np.random.uniform(low=-15, high=-5, size=(n_cl,)) * mvolt


# inhibition-specific parameters

taui = 10.*ms *mod # time constant of membrane inhibitory
tau2i = 2.*ms *mod

myuiMax = 1.05 *mod2
myuiMin = 1.0 *mod2

jEI = 0.045 / np.sqrt(scale)
jIE = 0.014 / np.sqrt(scale)
jII = 0.057 / np.sqrt(scale)

pII = 0.5
pEI = 0.5
pIE = 0.5


# Generate subpopulations

In [ ]:
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import as_strided


# add excitatory subpopulations
Gs, Ms = [], []
for i in range(n_cl):
    
    ns = {
         'Vt' : Vt,
         'Vr' : Vr,
         'tau' : taue,
         'tau1_i' : tau1,
         'tau2_i' : tau2i,
         'tau1_e' : tau1,
         'tau2_e' : tau2e,
         'normalization_e' : (tau1 - tau2e) / tau2e, 
         'invtau1_e' : 1./tau1,
         'invtau2_e' : 1./tau2e,
         'normalization_i' : (tau1 - tau2i) / tau2i, 
         'invtau1_i' : 1./tau1,
         'invtau2_i' : 1./tau2i           
          }    
    Gs.append(  NeuronGroup(N=Ns[i], 
                model=model_eqs,
                threshold='v > Vt',
                refractory=refractory,
                reset='v =Vr', 
                namespace=ns.copy()) )
    Gs[-1].myu = np.random.uniform(myueMin, myueMax, Ns[i])
    Gs[-1].v = ns['Vr'] + 1.*rand(Ns[i]) * (ns['Vt'] -ns['Vr'])    
    Ms.append( SpikeMonitor(Gs[i]) )
    
    
# add inhibitory subpopulation
    ns = {
         'Vt' : Vt,
         'Vr' : Vr,
         'tau' : taui,
         'tau1_i' : tau1,
         'tau2_i' : tau2i,
         'tau1_e' : tau1,
         'tau2_e' : tau2e,
         'normalization_e' : (tau1 - tau2e) / tau2e, 
         'invtau1_e' : 1./tau1,
         'invtau2_e' : 1./tau2e,
         'normalization_i' : (tau1-tau2i) / tau2i, 
         'invtau1_i' : 1./tau1,
         'invtau2_i' : 1./tau2i           
          }      
Gi = NeuronGroup(N=Ni, 
            model=model_eqs,
            threshold='v > Vt', 
            refractory=refractory,
            reset='v =Vr', 
            namespace=ns.copy())
Gi.v = ns['Vr'] + 1.*rand(Ni) * (ns['Vt'] -ns['Vr'])
Gi.myu = np.random.uniform(myuiMin, myuiMax, Ni)              
Ms.append( SpikeMonitor(Gi) )


# Wire up the populations

In [ ]:

print('adding EE synapses')

S = []
for i in range(n_cl):
    for j in range(n_cl):
        
        if i == j:
            """ think of something for here """
            ns= {'w' : type_type_strn[i,j] }
            S.append(Synapses(Gs[i], Gs[j], pre='y_i += w', namespace=ns.copy()))

            #base = np.random.random(Ns[i]) < type_type_conn[i,j] """ replace with randsample """
            base = np.sort(np.random.choice(Ns[i], np.round(Ns[i]*type_type_conn[i,j]), replace=False))
            full = as_strided(base, strides=base.strides+(0,), shape=base.shape+(Ns[j],) )

            idxi, idxj = np.where(full)[0], np.where(full)[1]
            #S[-1].connect(idxi, idxj)
            print(i, j, idxi.size, idxi.size/(1.0 * Ns[i] * Ns[j]))
            if idxi.size > 0:
                #S[-1].connect(True)
                S[-1].connect(idxi.tolist(), idxj.tolist())
            else:
                S[-1].connect(False)
        
        else:
            ns= {'w' : type_type_strn[i,j] }
            S.append(Synapses(Gs[i], Gs[j], pre='y_i += w', namespace=ns.copy()))

            #base = np.random.random(Ns[i]) < type_type_conn[i,j] """ replace with randsample """
            base = np.sort(np.random.choice(Ns[i], np.round(Ns[i]*type_type_conn[i,j]), replace=False))
            full = as_strided(base, strides=base.strides+(0,), shape=base.shape+(Ns[j],) )

            idxi, idxj = np.where(full)[0], np.where(full)[1]
            #S[-1].connect(idxi, idxj)
            print(i, j, idxi.size, idxi.size/(1.0 * Ns[i] * Ns[j]))
            if idxi.size > 0:
                #S[-1].connect(True)
                S[-1].connect(idxi.tolist(), idxj.tolist())
            else:
                S[-1].connect(False)

print('adding IE, EI, II synapses')
for i in range(n_cl):    
    ns= {'w' : jIE }
    S.append(Synapses(Gs[i], Gi, pre='y_e += w', namespace=ns.copy()))
    S[-1].connect(True, p = pIE)
for j in range(n_cl):
    ns= {'w' : jEI }
    S.append(Synapses(Gi, Gs[j],  pre='y_i += w', namespace=ns.copy()))
    S[-1].connect(True, p = pEI)
ns= {'w' : jII }
S.append(Synapses(Gi, Gi, pre='y_i += w', namespace=ns))
S[-1].connect(True, p = pII)        
    
Gs.append(Gi)

net = Network(Gs)
net.add(Ms)
net.add(S)

print('starting run')

net.run(duration = duration, namespace={})

print('done')
    

# Plot

In [ ]:
plt.figure(1)
plt.subplot(1,2,1)
plt.imshow(type_type_conn, interpolation='none')
plt.title('structural connectivity')
plt.xlabel('# group')
plt.ylabel('# group')
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(type_type_strn, interpolation='none')
plt.title('connectivity strength')
plt.xlabel('# group')
plt.colorbar()
plt.show()

In [ ]:
plt.figure(2)
NsAll = np.hstack((Ns, Ni))
for j in range(len(Ms)):
    plt.plot([0, duration], [NsAll[:j].sum(), NsAll[:j].sum()])
    plt.hold(True)
    for i in range(NsAll[j]):
        spktms = Ms[j].spike_trains()[i]/second
        plt.plot(spktms, i*np.ones(spktms.size)+NsAll[:j].sum(), '.')
plt.show()

# Store results

In [ ]:
np.savez('results/tmp', [Ms[j].spike_trains() for j in range(n_cl)], Ms[-1].spike_trains())

In [ ]:
import timeit

timeit.Timer?